# Predict New Data

In [1]:
from helper import generate_data

In [10]:
df_new_data = generate_data(20, seed=123)
df_new_data.drop(columns=['Risiko Stunting'], inplace=True)

In [11]:
from pycaret.classification import *
loaded_bestmodel = load_model('best-model')
print(loaded_bestmodel)

Transformation Pipeline and Model Successfully Loaded
Pipeline(memory=Memory(location=None),
         steps=[('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('trained_model',
                 LogisticRegression(max_iter=1000, random_state=6820))])


In [12]:
predicted_data = predict_model(loaded_bestmodel, data = df_new_data)

In [14]:
predicted_data.head()

,ID Anak,Usia (bulan),Berat Badan (kg),Tinggi Badan (cm),Asupan Gizi (Kcal/hari),Kualitas Air (%),Status Ekonomi,Akses Fasilitas Kesehatan,Riwayat Kesehatan,prediction_label,prediction_score
0,1,52,15.8,98.699997,1365,88,3,1,1,0,1.0
1,2,38,9.5,72.800003,997,75,2,0,1,1,1.0
2,3,53,15.8,105.300003,1498,90,3,1,1,0,1.0
3,4,31,12.6,82.500000,1340,96,3,1,1,0,1.0
4,5,25,12.9,102.900002,1376,91,3,1,1,0,1.0


# Pandas to Markdown

In [22]:
predicted_data.drop(columns=['prediction_score'], inplace=True)
predicted_data.rename(columns={'prediction_label': 'Risiko Stunting'}, inplace=True)
markdown_table = predicted_data.to_markdown()
print(markdown_table)

|    |   ID Anak |   Usia (bulan) |   Berat Badan (kg) |   Tinggi Badan (cm) |   Asupan Gizi (Kcal/hari) |   Kualitas Air (%) |   Status Ekonomi |   Akses Fasilitas Kesehatan |   Riwayat Kesehatan |   Risiko Stunting |
|---:|----------:|---------------:|-------------------:|--------------------:|--------------------------:|-------------------:|-----------------:|----------------------------:|--------------------:|------------------:|
|  0 |         1 |             52 |               15.8 |                98.7 |                      1365 |                 88 |                3 |                           1 |                   1 |                 0 |
|  1 |         2 |             38 |                9.5 |                72.8 |                       997 |                 75 |                2 |                           0 |                   1 |                 1 |
|  2 |         3 |             53 |               15.8 |               105.3 |                      1498 |                 9

# Generate Report

In [23]:
from openai import OpenAI
api_key = "sk-*"
client = OpenAI(api_key=api_key)


In [29]:
system_prompt = """You are a healthcare professional and an expert in the field. Your responsibility is "pencegah stunting & peningkatan gizi anak sekolah." You will create a report based on the predictive data on the "risiko stunting" for children that I provide. Here is an explanation of each column:
1. ID Anak: Identification number of the child.
2 .Usia (bulan): Age of the child in months.
3. Berat Badan (kg): Weight of the child in kilograms.
4. Tinggi Badan (cm): Height of the child in centimeters.
5. Asupan Gizi (Kcal/hari): Number of calories consumed by the child daily.
6. Kualitas Air (%): Percentage of access to clean and proper water.
7. Status Ekonomi: Economic category of the child’s family (1: rendah, 2: sedang, 3: tinggi).
8. Akses Fasilitas Kesehatan: Indicates if the child has access to good healthcare facilities (1: baik, 0: buruk).
9. Riwayat Kesehatan: Health history of the child (1: baik, 0: buruk).
10. Risiko Stunting: Target label indicating if the child is at risk of stunting (1: ya, 0: tidak).

The following points should be presented in the report:
1. Explanation of the Stunting Risk Prediction Results.
2. Risk Factor Analysis.
3. Recommendations for Stunting Prevention based on the analysis.
4. Conclusion.
"""

user_prompt = f"Berikut data dari daerah X\n{markdown_table}"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.9
)

# Print Report

In [37]:
import textwrap

wrapped_text = textwrap.fill(chat_completion.choices[0].message.content, width=200, replace_whitespace=False)
print(wrapped_text)

# Laporan Risiko Stunting Anak di Daerah X

## 1. Penjelasan Hasil Prediksi Risiko Stunting
Hasil prediksi menunjukkan bahwa dari 20 anak yang dianalisis, terdapat 8 anak yang berisiko stunting (40%).
Prediksi risiko stunting didasarkan pada faktor-faktor seperti usia, berat badan, tinggi badan, asupan gizi, kualitas air, status ekonomi, akses fasilitas kesehatan, dan riwayat kesehatan. Anak-anak
yang memiliki kekurangan berat badan dan tinggi badan relatif di bawah standar pertumbuhan, serta asupan gizi yang kurang memadai, lebih cenderung berada dalam kategori risiko stunting.

## 2.
Analisis Faktor Risiko
Berdasarkan data yang terkumpul, beberapa faktor yang menunjukkan pengaruh signifikan terhadap risiko stunting antara lain:
- **Usia**: Anak-anak yang lebih muda (di bawah 2
tahun) menunjukkan risiko yang lebih tinggi terhadap stunting.
- **Berat Badan dan Tinggi Badan**: Anak-anak yang memiliki berat dan tinggi badan yang rendah dibandingkan dengan standar pertumbuhan
memiliki ris